In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import json
import re

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)

def generate_features(product_name: str, brand: str):
    prompt = f"""
List the top 3 features of the product "{product_name}" (brand: {brand})
and provide a short usage example. Return strictly in JSON with keys:
"feature1", "feature2", "feature3", "usage_example".
"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=150)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract JSON
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if match:
        try:
            return json.loads(match.group(0))
        except json.JSONDecodeError:
            pass

    # fallback
    return {"feature1": "", "feature2": "", "feature3": "", "usage_example": text}

# Test
report = generate_features("OnePlus Nord CE 2 Lite", "OnePlus")
print(report)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'feature1': '5G connectivity', 'feature2': '6.7 inch AMOLED display', 'feature3': '48MP triple rear camera', 'usage_example': 'With 5G connectivity, you can stream high-quality videos and play online games without any buffering or lag. The 6.7 inch AMOLED display provides a vivid and immersive viewing experience, making it perfect for watching movies or playing games. The 48MP triple rear camera allows you to capture stunning photos and videos, even in low light conditions.'}
